# RAG Evaluation : Eval Dataset Generation

## Build a synthetic dataset for evaluation

If we do not have access to a real Q&A datset, we can use LLMs to generate synthetic Question and Answer couples based on a given context. This method is a very common approach, the well-known RAGAS package is based on synthetic Q&A. Furthermore, the zero-shot or few-short learning LLM abilities allow to filter out these generated proposals by asking (with a specific prompt) to evaluate (give a grade between 1 and 5) human understandable criteria (like groundedness, relevence,...) in a specific format like JSON. 

In [ ]:
import pandas as pd

data = pd.read_csv(
    "/home/onyxia/work/llm-open-data-insee/data_complete.csv", low_memory=False
)  # we assume the textual information have already been extracted.

In [ ]:
from db_building import extract_paragraphs

results = extract_paragraphs(data)  # gather textual information from the same page together

In [ ]:
ds = pd.DataFrame.from_dict(results)
ds.to_csv("insee_documents.csv", index=False)

In [ ]:
import pandas as pd
from langchain.docstore.document import Document as LangchainDocument
from tqdm import tqdm

ds = pd.read_csv("insee_documents.csv")
langchain_docs = [
    LangchainDocument(
        page_content=doc["document"],
        metadata={"source": doc["url_source"], "title": doc["title"], "insee_id": doc["id_origin"]},
    )
    for _, doc in tqdm(ds.iterrows())
]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,  # define the size of the chunck
    chunk_overlap=200,  # define the overlapping between following chunks
    add_start_index=True,
    separators=["\n\n", "\n", ".", " ", ""],
)

docs_processed = []
for doc in tqdm(langchain_docs):
    docs_processed += text_splitter.split_documents([doc])

In [ ]:
len(docs_processed)

Note : Phi3 microsoft SLM is not yet in HF official package version. Loading a Development version is required. 

In [ ]:
#!pip uninstall -y transformers && pip install git+https://github.com/huggingface/transformers
import torch
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# model_name = "microsoft/Phi-3-mini-128k-instruct" #(to download)
# path_model = "microsoft-Phi-3-mini-128k-instruct" #(to load)
# model.save_pretrained("microsoft-Phi-3-mini-128k-instruct")

model_name = "microsoft/Phi-3-mini-4k-instruct"  # smaller model

# load LLM config
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
# config.max_position_embeddings = 8096
# load quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=config,
    quantization_config=quantization_config,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
model

The aim is to generate **questions** and **answers** based on large **context** (part of INSEE documents). 

In [ ]:
QA_generation_prompt = """
<|user|>
Ta tâche consiste à écrire une question factuelle et sa réponse en te basant sur un contexte donné.
Ta question factuelle doit pouvoir être répondue par une information factuelle spécifique et concise tirée du contexte.
Ta question factuelle doit être formulée dans le même style que les questions que les utilisateurs pourraient poser dans un moteur de recherche.
Cela signifie que ta question factuelle NE DOIT PAS mentionner des phrases comme "selon le passage" ou "le contexte".
<|user|>
<|assistant|>
Voici maintenant le contexte.

Contexte : {context}

Question factuelle : (ta question factuelle)
Réponse : (ta réponse à la question factuelle)
<|assistant|>
"""

In [ ]:
import numpy as np

cont_sam = docs_processed[int(np.random.choice(range(len(docs_processed)), 1)[0])]
print(QA_generation_prompt.format(context=cont_sam.page_content))

In [ ]:
from transformers import pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
pipe.device

In [ ]:
def clean_output(s):
    """remove unwanted characters"""
    return s.replace("\n", "")

In [ ]:
import random
import time

from transformers import pipeline

generation_args = {
    "max_new_tokens": 300,
    "return_full_text": False,
    "do_sample": False,
}

import torch


def generate_test_dataset(pipeline, docs_processed, N_GENERATIONS, BATCH_SIZE):
    torch.cuda.empty_cache()
    print(f"Generating {N_GENERATIONS} QA couples...")
    t0 = time.time()
    outputs = []
    docs_sampled = random.sample(docs_processed, N_GENERATIONS)
    # Process documents in batches
    batch_contexts = [doc.page_content for doc in docs_sampled]
    # Generate QA couples for the batch
    generation_args["batch_size"] = BATCH_SIZE
    batch_prompts = [QA_generation_prompt.format(context=ctx) for ctx in batch_contexts]

    with torch.no_grad():
        generated_out = pipeline(batch_prompts, **generation_args)

    for output, sampled_context in zip(generated_out, docs_sampled):
        output_QA_couple = output[0]["generated_text"]
        question = output_QA_couple.split("Question factuelle : ")[-1].split("Réponse : ")[0]
        answer = output_QA_couple.split("Réponse : ")[-1]

        outputs.append(
            {
                "context": sampled_context.page_content,
                "question": clean_output(question),
                "answer": clean_output(answer),
                "source_doc": sampled_context.metadata["source"],
            }
        )
    print("run in ", time.time() - t0, "s")

    return outputs

In [ ]:
outputs = generate_test_dataset(pipe, docs_processed, N_GENERATIONS=200, BATCH_SIZE=32)

In [ ]:
display(pd.DataFrame(outputs).head(5))

In [ ]:
# SSP Cloud access : mc cp s3/projet-llm-insee-open-data/data/eval_data/q_and_a_insee_200.csv /home/onyxia/work/llm-open-data-insee/data/test
q_and_a_data = pd.DataFrame(outputs)
q_and_a_data.to_csv("q_and_a_insee_200.csv")

### Setup critique agents 
Need to evaluate if the generated questions are relevant
- Groundedness 
- Relevance
- Stand-alone question : Can the question be answerable without the linked document. (Is it a general knowledge question)

These 3 tests are done using prompting and asking the model to share its explanations and then its score. 
The scale is between 1 and 5.
At the end, the remaining questions received 4+

In [ ]:
q_and_a_data.head()

In [ ]:
question_groundedness_critique_prompt = """
<|user|>
Vous allez recevoir un contexte et une question.
Votre tâche est de fournir une évaluation globale notant dans quelle mesure on peut répondre de manière non ambiguë à la question donnée avec le contexte donné.
Donnez votre réponse sur une échelle de 1 à 5, où 1 signifie que la question n'est pas du tout répondable compte tenu du contexte, et 5 signifie que la question est clairement et sans ambiguïté répondable avec le contexte.

Fournissez votre réponse comme suit :

Évaluation : (votre justification de la note, sous forme de texte)
Note totale : (votre note, entre 1 et 5)

Vous DEVEZ fournir des valeurs pour 'Évaluation :' et 'Note totale :' dans votre réponse et rien d'autre. 

Maintenant, voici la question et le contexte.
Question : {question}
Contexte : {context}
<|end|>
<|assistant|>
"""
question_relevance_critique_prompt = """
<|user|>
Vous allez recevoir une question.
Votre tâche est de fournir une 'notation globale' représentant à quel point cette question peut être utile pour les agents de l'institut de statistique public français.
Donnez votre réponse sur une échelle de 1 à 5, où 1 signifie que la question n'est pas du tout utile, et 5 signifie que la question est extrêmement utile.

Fournissez votre réponse comme suit :

Évaluation : (votre justification de la note, sous forme de texte)
Note totale : (votre note, sous forme d'un nombre entre 1 et 5)

Vous DEVEZ fournir des valeurs pour 'Évaluation :' et 'Note totale :' dans votre réponse.

Maintenant, voici la question.

Question : {question}
<|end|>
<|assistant|>
"""

question_standalone_critique_prompt = """ 
<|user|>
Vous allez recevoir une question.
Votre tâche est de fournir une 'notation globale' représentant à quel point cette question est indépendante du contexte.
Donnez votre réponse sur une échelle de 1 à 5, où 1 signifie que la question dépend d'informations supplémentaires pour être comprise, et 5 signifie que la question a du sens par elle-même.
Par exemple, si la question fait référence à un cadre particulier, comme 'dans le contexte' ou 'dans le document', la note doit être de 1.
Les questions peuvent contenir des noms techniques obscurs ou des acronymes comme INSEE ou du vocabulaire propre au statistiques et obtenir tout de même une note de 5 : il suffit simplement qu'un agent d'un institut de statistique public ayant accès à la documentation comprenne de quoi parle la question.

Fournissez votre réponse comme suit :

Évaluation : (votre justification de la note, sous forme de texte)
Note totale : (votre note, sous forme d'un nombre entre 1 et 5)

Vous DEVEZ fournir des valeurs pour 'Évaluation :' et 'Note totale :' dans votre réponse.

Maintenant, voici la question.

Question : {question}\n
<|end|>
<|assistant|>
"""

In [ ]:
import re

import pandas as pd
from tqdm import tqdm


def extract_btw_tag(text, tag_1, tag_2):
    pattern = f"{tag_1}(.*?){tag_2}"
    return re.findall(pattern, text)[0] if re.findall(pattern, text) else None


def extract_score_eval(text):
    eval_text = extract_btw_tag(text, tag_1="Évaluation : ", tag_2="\n")
    score_text = extract_btw_tag(text, tag_1="Note totale : ", tag_2="\n")
    score = int(score_text) if score_text else None
    return score, eval_text


def critique_Q_and_A(pipeline, dataset, args, k=2):
    data = dataset.copy()

    num_rows = len(data)

    for batch_start in tqdm(range(0, num_rows, k)):
        batch_end = min(
            batch_start + k, num_rows
        )  # Adjust batch end to handle the last incomplete batch
        batch_prompts = []

        for idx in range(batch_start, batch_end):
            row = data.iloc[idx]
            batch_prompts.extend(
                [
                    question_groundedness_critique_prompt.format(
                        context=row.context, question=row.question
                    ),
                    question_relevance_critique_prompt.format(question=row.question),
                    question_standalone_critique_prompt.format(question=row.question),
                ]
            )

        outputs = pipeline(batch_prompts, **args)

        try:
            for j, idx in enumerate(range(batch_start, batch_end)):
                metrics = {}
                metrics["groundedness_score"], metrics["groundedness_eval"] = extract_score_eval(
                    outputs[3 * j][0]["generated_text"]
                )
                metrics["relevance_score"], metrics["relevance_eval"] = extract_score_eval(
                    outputs[3 * j + 1][0]["generated_text"]
                )
                metrics["standalone_score"], metrics["standalone_eval"] = extract_score_eval(
                    outputs[3 * j + 2][0]["generated_text"]
                )

                data.loc[idx, list(metrics.keys())] = list(metrics.values())

        except Exception as e:
            print("Error:", e)
            continue

    return data

In [ ]:
row = q_and_a_data.iloc[0]
prompt_final = question_groundedness_critique_prompt.format(
    context=row.context, question=row.question
)
prompt_final

In [ ]:
generated_questions = critique_Q_and_A(pipe, q_and_a_data, generation_args, k=10)

In [ ]:
generated_questions.head(1)

Filter out bad questions based on our critique agent scores

In [ ]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

print("Evaluation dataset before filtering:")

display(
    generated_questions[
        [
            "question",  # question generated based on given context
            "answer",  # answer generated based on given context (and generated question)
            "groundedness_score",  # score
            "relevance_score",  # score
            "standalone_score",  # score
        ]
    ]
)

# Select only the most relevant questions
generated_questions = generated_questions.loc[
    (generated_questions["groundedness_score"] >= 4)
    & (generated_questions["relevance_score"] >= 4)
    & (generated_questions["standalone_score"] >= 4)
]
print("number of selected Q&A couple : ", len(generated_questions))

print("============================================")
print("Final evaluation dataset:")
display(
    generated_questions[
        [
            "question",
            "answer",
            "groundedness_score",
            "relevance_score",
            "standalone_score",
        ]
    ]
)

In [ ]:
generated_questions.to_csv("eval_dataset.csv", index=False)

In [ ]:
!mc cp /home/onyxia/work/eval_dataset.csv s3/projet-llm-insee-open-data/data/eval_data/